In [1]:
# Set the working directory to the parent directory
import sys
sys.path.append('..')
sys.dont_write_bytecode = True

import os
os.chdir("..")

# Import relevant packages
from src.eda import data_info
from src.models import AutoencoderTrainer, AnomalyDetector, AutoencoderTuner
from src.explainability import ShapKernelExplainer

# Import necessary libraries
import shap
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import pandas as pd

In [ ]:
dpsgd_versions = pd.read_csv('experiments/perf_summary/dpsgd_val_results.csv')
dpsgd_explainer = ShapKernelExplainer(model_type='dpsgd')
for version in dpsgd_versions["version"].tolist():
    detector, noise_multiplier = dpsgd_explainer.version_info_extract(version=version)
    explain_with_kernelshap = dpsgd_explainer.explain_with_kernelshap(detector, noise_multiplier=noise_multiplier)
    print("Kernel SHAP explanation completed.")

0     202505070419
1     202505070955
2     202505071004
3     202505071236
4     202505071332
5     202505111328
6     202505110210
7     202505102039
8     202505101605
9     202505070326
10    202505070348
11    202505070407
Name: version, dtype: int64